<a href="https://colab.research.google.com/github/awke4/Kidney_Glom_seg_maskrcnn_TF2/blob/main/kidney_glomerulus_seg_maskrcnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kidney Glomerulus Segmentation with Mask R-CNN (TF2)


---



這個 Notebook 示範完整流程：

1. 安裝舊版keras2
2. 下載專案（從 GitHub clone）
3. 安裝必要套件（不動 Colab 內建 TensorFlow）
4. 使用 `data_demo` 做 debug / 推論 / 評估 (用demo dataset作流程展示)
5. 使用完整 dataset 做訓練 / 推論 / 評估（礙於colab效能，時間過長，可跳過）
6. 顯示 segmentation 結果與混淆矩陣

> 專案 GitHub：  
> https://github.com/awke4/Kidney_Glom_seg_maskrcnn_TF2.git






In [ ]:
# 1. 安裝舊版 Keras2 (tf_keras)
!pip install -q tf-keras==2.18.0

# 2. 一定要在 import tensorflow 之前設好環境變數
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1"

# 3. 再來才 import tensorflow / keras
import tensorflow as tf
from tensorflow import keras

print("TF version:", tf.__version__)
print("tf.keras module:", keras.__name__)
print("tf.keras loaded from:", keras.__file__)

# 可選：直接看 tf_keras 本尊的版本
import tf_keras
print("tf_keras version:", tf_keras.__version__)




In [ ]:
# Clone GitHub 專案
# 永遠先回到 /content，再刪掉舊的資料夾，再重新 clone
%cd /content
!rm -rf Kidney_Glom_seg_maskrcnn

!git clone https://github.com/awke4/Kidney_Glom_seg_maskrcnn_TF2.git
%cd Kidney_Glom_seg_maskrcnn_TF2
!ls


In [ ]:
# 安裝必要套件（不安裝 TF/keras）
!pip install imgaug opencv-python opencv-contrib-python scikit-image matplotlib pillow tqdm

In [ ]:
# 查看資料夾結構（確認 demo dataset 有在）
import os, textwrap
%cd /content/Kidney_Glom_seg_maskrcnn_TF2
print("目前目錄：", os.getcwd())
print("\n專案內容：")
for item in os.listdir("."):
    print(" -", item)

print("\ndata_demo 內容：")
for root, dirs, files in os.walk("data_demo"):
    level = root.count(os.sep) - "data_demo".count(os.sep)
    indent = "  " * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = "  " * (level + 1)
    for f in files:
        print(f"{subindent}{f}")


In [ ]:
# 用 demo dataset 做 debug（確認 Dataset 類別沒問題）
%cd /content/Kidney_Glom_seg_maskrcnn_TF2
!PYTHONPATH=$(pwd) python samples/kidney_glom/kidney_glom.py debug --dataset data_demo

In [ ]:
# 設定路徑變數
# Demo dataset（小資料，主要用來測試 pipeline）
DEMO_DATASET_DIR = "data_demo"

# 完整資料集：
FULL_DATASET_DIR = "datasets/kidney_glom"

# Log 目錄，用來存訓練好的模型
LOGS_DIR = "logs_kidney_tf2"


In [ ]:
# 用「demo datasets」訓練（礙於colab限制，先不用完整datasets）
# 由於coco權重格式與 TF2.18+Keras3 相容性問題，先採取隨機權重
# --weights coco \  => --weights random \
import os

if os.path.isdir(DEMO_DATASET_DIR):
    print("使用小型資料集訓練：", DEMO_DATASET_DIR)
    !python samples/kidney_glom/kidney_glom.py train \
        --dataset {DEMO_DATASET_DIR} \
        --weights random \
        --logs {LOGS_DIR}
else:
    print("找不到資料集資料夾：", DEMO_DATASET_DIR)
    print("請確認 DEMO_DATASET_DIR 參數路徑正確")


In [ ]:
# 自動尋找最新的權重檔（從 logs 目錄）
import glob

def find_last_weights(logs_dir=LOGS_DIR, prefix="kidney_glom"):
    if not os.path.isdir(logs_dir):
        return None

    subdirs = [d for d in os.listdir(logs_dir) if d.startswith(prefix)]
    if not subdirs:
        return None
    subdirs.sort()
    last_dir = os.path.join(logs_dir, subdirs[-1])

    pattern = os.path.join(last_dir, "mask_rcnn_kidney_glom_*.h5")
    candidates = glob.glob(pattern)
    if not candidates:
        return None
    candidates.sort()
    return candidates[-1]
'''
權重可使用:
    1.剛剛訓練出的
    2.results/weight/mask_rcnn_kidney_glom_0030.h5
'''
#weights_path = find_last_weights()
weights_path = "results/weight/mask_rcnn_kidney_glom_0030.h5"
print("找到的最新權重：", weights_path)


In [ ]:
# 做批次推論
'''
推論對象可以是:
    1.FULL_DATASET_DIR
    2.DEMO_DATASET_DIR
=> 結果存在對應的資料夾中
'''
if weights_path is None:
    print("目前沒有權重檔，請先完成訓練或上傳 .h5。")
else:
    print("使用權重：", weights_path)
    !python samples/kidney_glom/batch_infer_kidney_glom.py \
        --dataset {DEMO_DATASET_DIR} \
        --weights {weights_path}


In [ ]:
# 顯示推論結果圖片
import matplotlib.pyplot as plt
from pathlib import Path

result_dir = Path(DEMO_DATASET_DIR) / "results"
print("結果資料夾：", result_dir)

if not result_dir.is_dir():
    print("找不到 results 資料夾，可能還沒跑 batch_infer。")
else:
    images = sorted(result_dir.glob("*.png"))
    print("找到結果圖張數：", len(images))
    for img_path in images:
        img = plt.imread(img_path)
        plt.figure(figsize=(6, 6))
        plt.imshow(img)
        plt.title(img_path.name)
        plt.axis("off")
        plt.show()


In [ ]:
# 用 demo dataset 做評估（Dice / F1 / 混淆矩陣）
if weights_path is None:
    print("目前沒有權重檔，請先完成訓練或上傳 .h5。")
else:
    !python samples/kidney_glom/evaluate_kidney_glom.py \
        --dataset {FULL_DATASET_DIR} \
        --weights {weights_path} \
        --iou 0.5


In [ ]:
# 顯示混淆矩陣
import matplotlib.pyplot as plt
from pathlib import Path

cm_path = Path(DEMO_DATASET_DIR) / "confusion_matrix.png"

if cm_path.is_file():
    img = plt.imread(cm_path)
    plt.figure(figsize=(5, 5))
    plt.imshow(img)
    plt.axis("off")
    plt.title("Confusion Matrix (demo dataset)")
    plt.show()
else:
    print("找不到混淆矩陣圖片：", cm_path)
